<a href="https://colab.research.google.com/github/po3rin/python_playground/blob/master/torchnlp/try_sentence_bert_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
%%capture
!pip install transformers

!pip install tensorboard

!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
!pip install fugashi ipadic

In [30]:
import torch
from torch.nn.utils.rnn import pad_sequence

from transformers import BertJapaneseTokenizer
from transformers.models.bert_japanese import tokenization_bert_japanese
from transformers import BertModel, BertConfig


In [31]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [32]:
tknz = BertJapaneseTokenizer(vocab_file="drive/MyDrive/resources/sentence-bert/vocab.txt", do_lower_case=False, do_basic_tokenize=False)
tknz.word_tokenizer = tokenization_bert_japanese.MecabTokenizer()

In [33]:
config = BertConfig.from_json_file("drive/MyDrive/resources/sentence-bert/config.json")
net = BertModel.from_pretrained("drive/MyDrive/resources/sentence-bert/pytorch_model.bin", config=config).to(device)

In [34]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


In [35]:
ss = [
      "大学構内での喫煙は禁止です。",
      "学校でタバコを吸うのはダメです。",
      "今日は学校でタバコを買った。"
]

In [36]:
x = torch.LongTensor(ids).unsqueeze(0)
a = net(x)

bs, wc, dm = a[0].shape
sum_vec = a[0][0][0]
for i in range(1, wc):
  sum_vec += a[0][0][i]
mv = sum_vec /wc
mv.shape

torch.Size([768])

In [37]:
xs1, xmsk = [], []
for i in range(len(ss)):
    tid = tknz.encode(ss[i])
    xs1.append(torch.LongTensor(tid))
    xmsk.append(torch.LongTensor([1] * len(tid)))
xs1 = pad_sequence(xs1, batch_first=True).to(device)
xmsk = pad_sequence(xmsk, batch_first=True).to(device)

out = net(xs1,xmsk)
sv = mean_pooling(out, xmsk)
print("cos(s0,s1) = ", torch.cosine_similarity(sv[0], sv[1], dim=0).item())
print("cos(s0,s2) = ", torch.cosine_similarity(sv[0], sv[2], dim=0).item())
print("cos(s1,s2) = ", torch.cosine_similarity(sv[1], sv[2], dim=0).item())


cos(s0,s1) =  0.5350393056869507
cos(s0,s2) =  0.18858887255191803
cos(s1,s2) =  0.6100609302520752
